In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from KuhnPoker import KuhnPokerModel, KuhnPokerInfoSet, Card
from TreeVisualizer import plot_trees

import numpy as np
import pickle
import ipywidgets as widgets
from IPython.display import display


np.set_printoptions(precision=3)

In [15]:
with open(f'debug/tree_snapshots.pkl', 'rb') as file:
    trees_hist = pickle.load(file)

In [16]:
slider = widgets.IntSlider(min=0, max=len(trees_hist)-1, step=1, description='Hist Index')

def update_plot(index):
    plot_trees(trees_hist[index])
    
widgets.interactive(update_plot, index=slider)

interactive(children=(IntSlider(value=0, description='Hist Index', max=1000), Output()), _dom_classes=('widget…